# Data Quality Analysis & Cleaning

Dieses Notebook widmet sich der systematischen Prüfung und Bereinigung der Datenqualität.
Basierend auf den Erkenntnissen der EDA werden hier Maßnahmen ergriffen, um einen sauberen Datensatz für nachfolgende Analysen zu erstellen.

**Ziele:**
- Behandlung fehlender Werte (Missing Values)
- Entfernung von Duplikaten
- Bereinigung inkonsistenter Daten (z.B. negative Mengen, Preise <= 0)
- Speichern des bereinigten Datensatzes als `retail_clean.parquet`

In [1]:
from pathlib import Path
import pandas as pd
import numpy as np

# Pfade definieren
PROJECT_ROOT = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()
DATA_PATH = PROJECT_ROOT / "data" / "processed" / "retail_raw.parquet"
OUTPUT_PATH = PROJECT_ROOT / "data" / "processed" / "retail_clean.parquet"

print("Lade Daten von:", DATA_PATH)
df = pd.read_parquet(DATA_PATH)
print("Original Shape:", df.shape)

Lade Daten von: C:\Users\admin\Desktop\AI Sec Project\GitHub\data-science-projects\1.1-ecommerce-analytics\data\processed\retail_raw.parquet
Original Shape: (541909, 8)


## 1. Missing Values

Wir prüfen auf fehlende Werte. Ein kritischer Punkt ist die `CustomerID`.
Zeilen ohne `CustomerID` können nicht für kundenbezogene Analysen (CLV, Cohorts) genutzt werden, sind aber für die reine Umsatzbetrachtung relevant.

In [2]:
missing_counts = df.isna().sum()
print("Fehlende Werte pro Spalte:")
print(missing_counts[missing_counts > 0])

# Entscheidung: Wir behalten Zeilen ohne CustomerID vorerst, markieren sie aber.
# Für spezifische Kundenanalysen müssen diese später gefiltert werden.
# Wir füllen NaN in Description mit 'Unknown'

df['Description'] = df['Description'].fillna('Unknown')
print("Missing Values nach Imputation in Description:", df['Description'].isna().sum())

Fehlende Werte pro Spalte:
CustomerID    135080
dtype: int64
Missing Values nach Imputation in Description: 0


## 2. Duplikate

Vollständige Duplikate verzerren die Analyse und sollten entfernt werden.

In [3]:
duplicates = df.duplicated().sum()
print(f"Anzahl Duplikate: {duplicates}")

if duplicates > 0:
    df = df.drop_duplicates()
    print(f"Duplikate entfernt. Neue Shape: {df.shape}")

Anzahl Duplikate: 5268


Duplikate entfernt. Neue Shape: (536641, 8)


## 3. Inkonsistenzen bereinigen

- **Negative Mengen**: Oft Retouren oder Stornos. Für die "Clean Sales" Analyse entfernen wir diese hier.
- **Preise <= 0**: Werbegeschenke oder Fehler. Entfernen.

*Hinweis: In einem Rohdaten-Warehouse würden wir diese Daten behalten und flags setzen. Für diesen Analyse-Datensatz filtern wir strikt.*

In [4]:
# Filterbedingungen
mask_quantity = df['Quantity'] > 0
mask_price = df['UnitPrice'] > 0

df_clean = df[mask_quantity & mask_price].copy()

print(f"Zeilen vor Filterung: {df.shape[0]}")
print(f"Zeilen nach Filterung (Quantity > 0 & Price > 0): {df_clean.shape[0]}")
print(f"Entfernte Zeilen: {df.shape[0] - df_clean.shape[0]}")

Zeilen vor Filterung: 536641
Zeilen nach Filterung (Quantity > 0 & Price > 0): 524878
Entfernte Zeilen: 11763


## 4. Speichern

Der bereinigte Datensatz wird als `retail_clean.parquet` gespeichert.

In [5]:
df_clean.to_parquet(OUTPUT_PATH)
print(f"Daten gespeichert unter: {OUTPUT_PATH}")

Daten gespeichert unter: C:\Users\admin\Desktop\AI Sec Project\GitHub\data-science-projects\1.1-ecommerce-analytics\data\processed\retail_clean.parquet